In [15]:
import cv2
import easyocr
from ultralytics import YOLO
from sort.sort import *
from util import get_car, read_license_plate, write_csv

In [13]:
# Object tracker
mot_tracker = Sort() 

In [7]:
coco_model = YOLO('weights/yolov8n.pt')
license_detector = YOLO('weights/best.pt')

In [14]:
results = {}

In [20]:
cap = cv2.VideoCapture('./sample.mp4')

vehicles = [2, 3, 5, 7]
frame_nmr = -1
results = {}

while cap.isOpened():
    frame_nmr += 1
    ret, frame = cap.read()

    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:
                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255,
                                                               cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
    else:
        break  # Break the loop if no more frames are available

# Release the video capture object
cap.release()

# write results
write_csv(results, './test.csv')


0: 384x640 21 cars, 1 bus, 2 trucks, 166.0ms
Speed: 5.0ms preprocess, 166.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 165.2ms
Speed: 5.0ms preprocess, 165.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 101.0ms
Speed: 4.1ms preprocess, 101.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 91.0ms
Speed: 3.6ms preprocess, 91.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 106.0ms
Speed: 5.0ms preprocess, 106.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 105.0ms
Speed: 5.0ms preprocess, 105.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 105.0ms
Speed: 3.0ms preprocess, 105.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Lice

In [21]:
import pandas as pd

df = pd.read_csv('test.csv')
df

,frame_nmr,car_id,car_bbox,license_plate_bbox,license_plate_bbox_score,license_number,license_number_score
0,2,5.0,[2197.0120693921162 1159.5715233530973 2787.44...,[2413.0380859375 1549.7369384765625 2586.33422...,0.535486,HU51VSU,0.132086
1,3,5.0,[2194.9118543113063 1164.9512974150027 2785.64...,[2412.140625 1551.341064453125 2588.2587890625...,0.572011,NI51TSU,0.032849
2,4,3.0,[749.8808570853806 1377.8957289709738 1429.912...,[949.166015625 1753.570068359375 1217.53820800...,0.589605,NA13NPU,0.394219
3,4,5.0,[2195.726371327958 1168.3553158898067 2786.239...,[2409.75732421875 1555.118408203125 2588.17797...,0.569501,NU51VSU,0.288748
4,5,5.0,[2194.369966370085 1172.3544796878173 2789.435...,[2407.999755859375 1564.0196533203125 2590.947...,0.580727,HU51VSU,0.373225
...,...,...,...,...,...,...,...
2211,3594,2624.0,[2254.8710633985775 1151.5248480061978 2759.24...,[2407.957275390625 1369.5029296875 2604.570800...,0.603412,SC56OYP,0.268265
2212,3595,2624.0,[2254.4009373709705 1150.614643758126 2758.587...,[2407.848388671875 1369.2587890625 2604.470214...,0.605714,SC56OYP,0.233665
2213,3596,2624.0,[2254.7688877219643 1149.9761938207776 2758.87...,[2416.7021484375 1375.107177734375 2604.565429...,0.604062,SC56DYP,0.687382
2214,3598,2624.0,[2256.3341633207883 1152.17374110039 2761.8206...,[2412.808837890625 1382.369140625 2603.4479980...,0.603337,SC56DYP,0.572310
